In [26]:
import sys
import xarray as xr
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime

In [27]:
def fix_time(ds):
    """If months are missing, fix the dataset by adding NaN values for those months."""

    var = list(ds.data_vars)[0]
    src = ds[var].encoding["source"]

    ds["time"] = ds["time"].astype("datetime64[ns]")

    # look for missing months and fill those variable values with NaN
    # get all months present in the time coordinate
    months_present = set(ds["time"].dt.month.values)
    all_months = set(range(1, 13))
    missing_months = sorted(list(all_months - months_present))

    if missing_months:
        print(
            f"Missing months in {src}: {missing_months} .... attempting to add all NaN array for these months."
        )
        # get the years present in the dataset
        years = np.unique(ds["time"].dt.year.values)
        # create new timestamps for each missing month in each year
        new_times = []
        for year in years:
            for month in missing_months:
                new_times.append(np.datetime64(f"{year}-{month:02d}-15"))

        # convert new_times to nanosecond precision to avoid xarray warning
        new_times = np.array(new_times, dtype="datetime64[M]").astype("datetime64[ns]")

        # reindex the variable to include the new times, filling missing values with NaN
        new_shape = (len(ds.lon), len(ds.lat), len(new_times))
        nan_data = xr.DataArray(
            np.full(new_shape, np.nan),
            dims=("lon", "lat", "time"),
            coords={"lon": ds.lon, "lat": ds.lat, "time": new_times},
        )
        new_ds = xr.Dataset({var: nan_data})
        combined = xr.concat([ds, new_ds], dim="time")
        combined = combined.sortby("time")

    return combined if missing_months else ds

In [28]:
dir = "/beegfs/CMIP6/jdpaul3/CMIP6_common_regrid/regrid/"
# output where the fixed files will be saved
output_dir = Path("/beegfs/CMIP6/jdpaul3/cmip6_regrid_timefix")

In [29]:
# list all monthly snw*.nc files in the directory and any subdirectories
files = list(Path(dir).rglob("snw_*mon_*.nc"))

In [30]:
# for each file, open it and add missing months, and save it to the output directory with the same directory structure
for file in files:
    try:
        ds = xr.open_dataset(file, decode_times=True)
        ds = fix_time(ds)
        output_file = output_dir / file.relative_to(dir)
        output_file.parent.mkdir(
            parents=True, exist_ok=True
        )  # Create parent directories if they don't exist
        ds.to_netcdf(output_file, mode="w", format="NETCDF4")
        ds.close()
        #print(f"Saved {file} successfully.")
    except Exception as e:
        print(f"Error processing {file}: {e}", file=sys.stderr)

Missing months in /beegfs/CMIP6/jdpaul3/CMIP6_common_regrid/regrid/TaiESM1/ssp370/LImon/snw/snw_LImon_TaiESM1_ssp370_regrid_201502-201512.nc: [1] .... attempting to add all NaN array for these months.
Missing months in /beegfs/CMIP6/jdpaul3/CMIP6_common_regrid/regrid/TaiESM1/ssp585/LImon/snw/snw_LImon_TaiESM1_ssp585_regrid_201502-201512.nc: [1] .... attempting to add all NaN array for these months.
Missing months in /beegfs/CMIP6/jdpaul3/CMIP6_common_regrid/regrid/NorESM2-MM/ssp245/LImon/snw/snw_LImon_NorESM2-MM_ssp245_regrid_201502-201512.nc: [1] .... attempting to add all NaN array for these months.
Missing months in /beegfs/CMIP6/jdpaul3/CMIP6_common_regrid/regrid/NorESM2-MM/ssp370/LImon/snw/snw_LImon_NorESM2-MM_ssp370_regrid_201502-201512.nc: [1] .... attempting to add all NaN array for these months.
Missing months in /beegfs/CMIP6/jdpaul3/CMIP6_common_regrid/regrid/NorESM2-MM/ssp126/LImon/snw/snw_LImon_NorESM2-MM_ssp126_regrid_201502-201512.nc: [1] .... attempting to add all NaN a

In [ ]:
'CESM2 E3SM-2-0 GFDL-ESM4 HadGEM3-GC31-MM MPI-ESM1-2-HR CNRM-CM6-1-HR EC-Earth3-Veg HadGEM3-GC31-LL MIROC6 MRI-ESM2-0'

